In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.context import SparkContext
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder.appName("Interview").getOrCreate()

24/04/24 01:31:58 WARN Utils: Your hostname, Shivanis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.106 instead (on interface en0)
24/04/24 01:31:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 01:31:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
''' Question 1. Write a PySpark Query using below input to get below output.  #KPMG
Input 
+-----+---------------------+
|  Name|             Hobbies|
+-----+---------------------+
| Alice|  Badmintion, Tennis|
|   Bob|     Tennis, Cricket|
| Julie|    Cricket, Carroms|
+-----+---------------------+
Output
+-----+------------+
|  Name|    Hobbies|
+-----+------------+
| Alice| Badmintion|
| Alice|     Tennis|
|   Bob|     Tennis|
|   Bob|    Cricket|
| Julie|    Cricket|
| Julie|    Carroms|
+-----+------------+

'''


data = [('Alice', 'Badminton, Tennis' ), ('Bob',  'Tennis, Circket'), ('Julie', 'Cricket, Carroms')]
columns = ["Name", "Hobbies"]
df1 = spark.createDataFrame(data, columns)
df1.show()

#split : In PySpark, the split() function is used to split a string column into an array of substrings based on a delimiter.
#explode : function is used to split a column that contains an array or map type into multiple rows, with each element of the array or map occupying its own row. 
from pyspark.sql.functions import split, explode
df1= df1.select(df1.Name, explode(split(df1.Hobbies, ',')).alias("Hobbies"))
df1.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=24482) is multi-threaded, use of fork() may lead to deadlocks in the child.


+-----+-----------------+
| Name|          Hobbies|
+-----+-----------------+
|Alice|Badminton, Tennis|
|  Bob|  Tennis, Circket|
|Julie| Cricket, Carroms|
+-----+-----------------+

+-----+---------+
| Name|  Hobbies|
+-----+---------+
|Alice|Badminton|
|Alice|   Tennis|
|  Bob|   Tennis|
|  Bob|  Circket|
|Julie|  Cricket|
|Julie|  Carroms|
+-----+---------+



In [19]:
'''Question2. Write a PySpark Query using below input to get below output
Input
+-------+-------+-------+
| City 1| City 2| City 3|
+-------+-------+-------+
|    Goa|       |     AP|
|       |     AP|   null|
|   null|       |   Bglu|
+-------+-------+-------+

Output
+-------+
| Result| 
+-------+
|    Goa|
|     AP| 
|   Bglu|
+-------+

'''

df2 = data = [('Goa', '', 'AP' ), ('',  'AP', None), (None, '', 'Bglu')]
columns = ["City1", "City2", "City3"]
df2 = spark.createDataFrame(data, columns)
df2.show()


#coalesce : In PySpark, the coalesce() function is used to return the first non-null value among the input columns. 
#It takes multiple columns as input and returns a new column with the first non-null value from those columns.
from pyspark.sql.functions import coalesce,when
df2 = df2.withColumn('Result', coalesce(
    when(df2.City1 == "", None).otherwise(df2.City1), 
    when(df2.City2 == "", None).otherwise(df2.City2), 
    when(df2.City3 == "", None).otherwise(df2.City3)
    )).drop(df2.City1, df2.City2, df2.City3)
df2.show()


+-----+-----+-----+
|City1|City2|City3|
+-----+-----+-----+
|  Goa|     |   AP|
|     |   AP| NULL|
| NULL|     | Bglu|
+-----+-----+-----+

+------+
|Result|
+------+
|   Goa|
|    AP|
|  Bglu|
+------+

